# Assembler rubustness - Contiguity

## Imports

In [1]:
import sys
from plotly.offline import plot
import glob
import fnmatch
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import json
import pandas as pd
from itertools import groupby
import csv
import numpy as np
from statistics import mean

## Global variables

In [2]:
METRICS_COLUMNS_GLOBAL = ['run','sample','assembler', 'contigs', 'basepairs', 'max_contig', 'Ns', 'n50', 'misassemblies', 'mapped_reads']
ASSEMBLER_PROCESS_LIST = ["BCALM2", "GATBMINIAPIPELINE", "MINIA", "MEGAHIT", "METASPADES", "UNICYCLER", "SPADES",
                          "SKESA", "VELVETOPTIMIZER", "IDBA"]
PROCESS_TO_NAME = {"BCALM2": "BCALM2", 
                   "GATBMINIAPIPELINE": "GATBMiniaPipeline",
                   "MINIA": "MINIA",
                   "MEGAHIT": "MEGAHIT", 
                   "METASPADES": "metaSPAdes", 
                   "UNICYCLER": "Unicycler", 
                   "SPADES": "SPAdes",
                    "SKESA": "SKESA",
                   "VELVETOPTIMIZER": "VelvetOptimizer",
                   "IDBA": "IDBA-UD"}
log_distributed = ['LHS', 'LNN', 'ERR2935805']
even_distribution = ['EMS', 'ENN', 'ERR2984773']
COLOURS = ['#2e83d1','#004B93', "#009392", "#39B185", "#9CCB86", "#E9E29C", "#EEB479", "#E88471","#CF597E", 'darkgray']

## Lx Metric

### Load data

In [10]:
_files = glob.glob('../Results/*/results/*/stats/*_lx.csv')

df_list = []
for f in _files:
    run = f.split('/')[-5]
    sample = f.split('/')[-3]
    assembler = f.split('/')[-1].split('_')[1]
    df = pd.read_csv(f)
    print(df)
    df['run'] = run
    df['Sample'] = sample
    df['Assembler'] = assembler
    df_list.append(df)

df_Lx = pd.concat(df_list)
df_Lx['distribution'] = np.where(df_Lx['Sample'].isin(log_distributed), 'Log', 'Even')
df_Lx

     Unnamed: 0                       Reference Assembler  Lx  nContigs
0             0               Bacillus_subtilis    SPAdes   0       0.0
1             1               Bacillus_subtilis    SPAdes   1       NaN
2             2               Bacillus_subtilis    SPAdes   2       NaN
3             3               Bacillus_subtilis    SPAdes   3       NaN
4             4               Bacillus_subtilis    SPAdes   4       NaN
..          ...                             ...       ...  ..       ...
127         127  Staphylococcus_aureus_plasmid3    SPAdes   6       NaN
128         128  Staphylococcus_aureus_plasmid3    SPAdes   7       NaN
129         129  Staphylococcus_aureus_plasmid3    SPAdes   8       NaN
130         130  Staphylococcus_aureus_plasmid3    SPAdes   9       NaN
131         131  Staphylococcus_aureus_plasmid3    SPAdes  10       NaN

[132 rows x 5 columns]
     Unnamed: 0                       Reference  Assembler  Lx  nContigs
0             0               Bacillus_

     Unnamed: 0                       Reference Assembler  Lx  nContigs
0             0               Bacillus_subtilis   IDBA-UD   0       0.0
1             1               Bacillus_subtilis   IDBA-UD   1       NaN
2             2               Bacillus_subtilis   IDBA-UD   2       NaN
3             3               Bacillus_subtilis   IDBA-UD   3       NaN
4             4               Bacillus_subtilis   IDBA-UD   4       NaN
..          ...                             ...       ...  ..       ...
127         127  Staphylococcus_aureus_plasmid3   IDBA-UD   6       NaN
128         128  Staphylococcus_aureus_plasmid3   IDBA-UD   7       NaN
129         129  Staphylococcus_aureus_plasmid3   IDBA-UD   8       NaN
130         130  Staphylococcus_aureus_plasmid3   IDBA-UD   9       NaN
131         131  Staphylococcus_aureus_plasmid3   IDBA-UD  10       NaN

[132 rows x 5 columns]
     Unnamed: 0                       Reference   Assembler  Lx  nContigs
0             0               Bacillus

     Unnamed: 0                       Reference Assembler  Lx  nContigs
0             0               Bacillus_subtilis    BCALM2   0       0.0
1             1               Bacillus_subtilis    BCALM2   1       NaN
2             2               Bacillus_subtilis    BCALM2   2       NaN
3             3               Bacillus_subtilis    BCALM2   3       NaN
4             4               Bacillus_subtilis    BCALM2   4       NaN
..          ...                             ...       ...  ..       ...
127         127  Staphylococcus_aureus_plasmid3    BCALM2   6       NaN
128         128  Staphylococcus_aureus_plasmid3    BCALM2   7       NaN
129         129  Staphylococcus_aureus_plasmid3    BCALM2   8       NaN
130         130  Staphylococcus_aureus_plasmid3    BCALM2   9       NaN
131         131  Staphylococcus_aureus_plasmid3    BCALM2  10       NaN

[132 rows x 5 columns]
     Unnamed: 0                       Reference  Assembler  Lx  nContigs
0             0               Bacillus_

,Unnamed: 0,Reference,Assembler,Lx,nContigs,run,Sample,distribution
0,0,Bacillus_subtilis,SPAdes,0,0.0,run3,LNN,Log
1,1,Bacillus_subtilis,SPAdes,1,NaN,run3,LNN,Log
2,2,Bacillus_subtilis,SPAdes,2,NaN,run3,LNN,Log
3,3,Bacillus_subtilis,SPAdes,3,NaN,run3,LNN,Log
4,4,Bacillus_subtilis,SPAdes,4,NaN,run3,LNN,Log
...,...,...,...,...,...,...,...,...
127,127,Staphylococcus_aureus_plasmid3,SKESA,6,NaN,run2,ERR2984773,Even
128,128,Staphylococcus_aureus_plasmid3,SKESA,7,NaN,run2,ERR2984773,Even
129,129,Staphylococcus_aureus_plasmid3,SKESA,8,NaN,run2,ERR2984773,Even
130,130,Staphylococcus_aureus_plasmid3,SKESA,9,NaN,run2,ERR2984773,Even


### Create Average, Upper bound and Lower bound variables for plot

In [8]:
plot_lx_df = pd.DataFrame(columns=['Reference', 'Assembler', 'Sample', "Average", "Upper bound", "Lower bound", 'Lx'])

for sample in sorted(df_Lx['Sample'].unique(), key=lambda v: v.upper(), reverse=True):
    for reference in sorted(df_Lx['Reference'].unique(), key=lambda v: v.upper(), reverse=True):
        for assembler in sorted(df_Lx['Assembler'].unique(), key=lambda v: v.upper(), reverse=True):
            for x in df_Lx['Lx'].unique():
                contigs = list(df_Lx['nContigs'][(df_Lx['Assembler'] == assembler) & (df_Lx['Reference'] == reference) & (df_Lx['Sample'] == sample) & (df_Lx['Lx'] == x)])
                if contigs:
                    avg = mean(contigs) if len(set(contigs)) > 1 else contigs[0]
                    upper_bound = max(contigs)
                    lower_bound = min(contigs)
                    plot_lx_df = plot_lx_df.append({'Reference':reference, 'Assembler': assembler, 'Sample': sample, "Average": avg, "Upper bound": upper_bound, "Lower bound": lower_bound, 'Lx': x}, ignore_index=True)

plot_lx_df

LNN Staphylococcus_aureus_plasmid3 VelvetOptimizer 0
[]
LNN Staphylococcus_aureus_plasmid3 VelvetOptimizer 1
[]
LNN Staphylococcus_aureus_plasmid3 VelvetOptimizer 2
[]
LNN Staphylococcus_aureus_plasmid3 VelvetOptimizer 3
[]
LNN Staphylococcus_aureus_plasmid3 VelvetOptimizer 4
[]
LNN Staphylococcus_aureus_plasmid3 VelvetOptimizer 5
[]
LNN Staphylococcus_aureus_plasmid3 VelvetOptimizer 6
[]
LNN Staphylococcus_aureus_plasmid3 VelvetOptimizer 7
[]
LNN Staphylococcus_aureus_plasmid3 VelvetOptimizer 8
[]
LNN Staphylococcus_aureus_plasmid3 VelvetOptimizer 9
[]
LNN Staphylococcus_aureus_plasmid3 VelvetOptimizer 10
[]
LNN Staphylococcus_aureus_plasmid3 Unicycler 0
[0.0, 0.0]
LNN Staphylococcus_aureus_plasmid3 Unicycler 1
[nan, nan]
LNN Staphylococcus_aureus_plasmid3 Unicycler 2
[nan, nan]
LNN Staphylococcus_aureus_plasmid3 Unicycler 3
[nan, nan]
LNN Staphylococcus_aureus_plasmid3 Unicycler 4
[nan, nan]
LNN Staphylococcus_aureus_plasmid3 Unicycler 5
[nan, nan]
LNN Staphylococcus_aureus_plasmid3 

LNN Staphylococcus_aureus_plasmid2 metaSPAdes 9
[nan, nan]
LNN Staphylococcus_aureus_plasmid2 metaSPAdes 10
[nan, nan]
LNN Staphylococcus_aureus_plasmid2 MEGAHIT 0
[0.0, 0.0]
LNN Staphylococcus_aureus_plasmid2 MEGAHIT 1
[nan, nan]
LNN Staphylococcus_aureus_plasmid2 MEGAHIT 2
[nan, nan]
LNN Staphylococcus_aureus_plasmid2 MEGAHIT 3
[nan, nan]
LNN Staphylococcus_aureus_plasmid2 MEGAHIT 4
[nan, nan]
LNN Staphylococcus_aureus_plasmid2 MEGAHIT 5
[nan, nan]
LNN Staphylococcus_aureus_plasmid2 MEGAHIT 6
[nan, nan]
LNN Staphylococcus_aureus_plasmid2 MEGAHIT 7
[nan, nan]
LNN Staphylococcus_aureus_plasmid2 MEGAHIT 8
[nan, nan]
LNN Staphylococcus_aureus_plasmid2 MEGAHIT 9
[nan, nan]
LNN Staphylococcus_aureus_plasmid2 MEGAHIT 10
[nan, nan]
LNN Staphylococcus_aureus_plasmid2 IDBA-UD 0
[0.0, 0.0]
LNN Staphylococcus_aureus_plasmid2 IDBA-UD 1
[nan, nan]
LNN Staphylococcus_aureus_plasmid2 IDBA-UD 2
[nan, nan]
LNN Staphylococcus_aureus_plasmid2 IDBA-UD 3
[nan, nan]
LNN Staphylococcus_aureus_plasmid2 IDBA-

[nan, nan]
LNN Staphylococcus_aureus_plasmid1 BCALM2 4
[nan, nan]
LNN Staphylococcus_aureus_plasmid1 BCALM2 5
[nan, nan]
LNN Staphylococcus_aureus_plasmid1 BCALM2 6
[nan, nan]
LNN Staphylococcus_aureus_plasmid1 BCALM2 7
[nan, nan]
LNN Staphylococcus_aureus_plasmid1 BCALM2 8
[nan, nan]
LNN Staphylococcus_aureus_plasmid1 BCALM2 9
[nan, nan]
LNN Staphylococcus_aureus_plasmid1 BCALM2 10
[nan, nan]
LNN Staphylococcus_aureus VelvetOptimizer 0
[]
LNN Staphylococcus_aureus VelvetOptimizer 1
[]
LNN Staphylococcus_aureus VelvetOptimizer 2
[]
LNN Staphylococcus_aureus VelvetOptimizer 3
[]
LNN Staphylococcus_aureus VelvetOptimizer 4
[]
LNN Staphylococcus_aureus VelvetOptimizer 5
[]
LNN Staphylococcus_aureus VelvetOptimizer 6
[]
LNN Staphylococcus_aureus VelvetOptimizer 7
[]
LNN Staphylococcus_aureus VelvetOptimizer 8
[]
LNN Staphylococcus_aureus VelvetOptimizer 9
[]
LNN Staphylococcus_aureus VelvetOptimizer 10
[]
LNN Staphylococcus_aureus Unicycler 0
[0.0, 0.0]
LNN Staphylococcus_aureus Unicycler 

LNN Salmonella_enterica MEGAHIT 3
[nan, nan]
LNN Salmonella_enterica MEGAHIT 4
[nan, nan]
LNN Salmonella_enterica MEGAHIT 5
[nan, nan]
LNN Salmonella_enterica MEGAHIT 6
[nan, nan]
LNN Salmonella_enterica MEGAHIT 7
[nan, nan]
LNN Salmonella_enterica MEGAHIT 8
[nan, nan]
LNN Salmonella_enterica MEGAHIT 9
[nan, nan]
LNN Salmonella_enterica MEGAHIT 10
[nan, nan]
LNN Salmonella_enterica IDBA-UD 0
[0.0, 0.0]
LNN Salmonella_enterica IDBA-UD 1
[nan, nan]
LNN Salmonella_enterica IDBA-UD 2
[nan, nan]
LNN Salmonella_enterica IDBA-UD 3
[nan, nan]
LNN Salmonella_enterica IDBA-UD 4
[nan, nan]
LNN Salmonella_enterica IDBA-UD 5
[nan, nan]
LNN Salmonella_enterica IDBA-UD 6
[nan, nan]
LNN Salmonella_enterica IDBA-UD 7
[nan, nan]
LNN Salmonella_enterica IDBA-UD 8
[nan, nan]
LNN Salmonella_enterica IDBA-UD 9
[nan, nan]
LNN Salmonella_enterica IDBA-UD 10
[nan, nan]
LNN Salmonella_enterica GATBMiniaPipeline 0
[0.0, 0.0]
LNN Salmonella_enterica GATBMiniaPipeline 1
[nan, nan]
LNN Salmonella_enterica GATBMinia

LNN Listeria_monocytogenes SPAdes 4
[nan, nan]
LNN Listeria_monocytogenes SPAdes 5
[nan, nan]
LNN Listeria_monocytogenes SPAdes 6
[nan, nan]
LNN Listeria_monocytogenes SPAdes 7
[nan, nan]
LNN Listeria_monocytogenes SPAdes 8
[nan, nan]
LNN Listeria_monocytogenes SPAdes 9
[nan, nan]
LNN Listeria_monocytogenes SPAdes 10
[nan, nan]
LNN Listeria_monocytogenes SKESA 0
[0.0, 0.0]
LNN Listeria_monocytogenes SKESA 1
[nan, nan]
LNN Listeria_monocytogenes SKESA 2
[nan, nan]
LNN Listeria_monocytogenes SKESA 3
[nan, nan]
LNN Listeria_monocytogenes SKESA 4
[nan, nan]
LNN Listeria_monocytogenes SKESA 5
[nan, nan]
LNN Listeria_monocytogenes SKESA 6
[nan, nan]
LNN Listeria_monocytogenes SKESA 7
[nan, nan]
LNN Listeria_monocytogenes SKESA 8
[nan, nan]
LNN Listeria_monocytogenes SKESA 9
[nan, nan]
LNN Listeria_monocytogenes SKESA 10
[nan, nan]
LNN Listeria_monocytogenes MINIA 0
[0.0, 0.0]
LNN Listeria_monocytogenes MINIA 1
[nan, nan]
LNN Listeria_monocytogenes MINIA 2
[nan, nan]
LNN Listeria_monocytogene

...

EMS Pseudomonas_aeruginosa SPAdes 8
[nan, nan]
EMS Pseudomonas_aeruginosa SPAdes 9
[nan, nan]
EMS Pseudomonas_aeruginosa SPAdes 10
[nan, nan]
EMS Pseudomonas_aeruginosa SKESA 0
[0.0, 0.0]
EMS Pseudomonas_aeruginosa SKESA 1
[51.0, 51.0]
EMS Pseudomonas_aeruginosa SKESA 2
[nan, nan]
EMS Pseudomonas_aeruginosa SKESA 3
[nan, nan]
EMS Pseudomonas_aeruginosa SKESA 4
[nan, nan]
EMS Pseudomonas_aeruginosa SKESA 5
[nan, nan]
EMS Pseudomonas_aeruginosa SKESA 6
[nan, nan]
EMS Pseudomonas_aeruginosa SKESA 7
[nan, nan]
EMS Pseudomonas_aeruginosa SKESA 8
[nan, nan]
EMS Pseudomonas_aeruginosa SKESA 9
[nan, nan]
EMS Pseudomonas_aeruginosa SKESA 10
[nan, nan]
EMS Pseudomonas_aeruginosa MINIA 0
[0.0, 0.0]
EMS Pseudomonas_aeruginosa MINIA 1
[nan, nan]
EMS Pseudomonas_aeruginosa MINIA 2
[nan, nan]
EMS Pseudomonas_aeruginosa MINIA 3
[nan, nan]
EMS Pseudomonas_aeruginosa MINIA 4
[nan, nan]
EMS Pseudomonas_aeruginosa MINIA 5
[nan, nan]
EMS Pseudomonas_aeruginosa MINIA 6
[nan, nan]
EMS Pseudomonas_aeruginosa 

EMS Listeria_monocytogenes BCALM2 7
[nan, nan]
EMS Listeria_monocytogenes BCALM2 8
[nan, nan]
EMS Listeria_monocytogenes BCALM2 9
[nan, nan]
EMS Listeria_monocytogenes BCALM2 10
[nan, nan]
EMS Lactobacillus_fermentum VelvetOptimizer 0
[0.0, 0.0]
EMS Lactobacillus_fermentum VelvetOptimizer 1
[nan, nan]
EMS Lactobacillus_fermentum VelvetOptimizer 2
[nan, nan]
EMS Lactobacillus_fermentum VelvetOptimizer 3
[nan, nan]
EMS Lactobacillus_fermentum VelvetOptimizer 4
[nan, nan]
EMS Lactobacillus_fermentum VelvetOptimizer 5
[nan, nan]
EMS Lactobacillus_fermentum VelvetOptimizer 6
[nan, nan]
EMS Lactobacillus_fermentum VelvetOptimizer 7
[nan, nan]
EMS Lactobacillus_fermentum VelvetOptimizer 8
[nan, nan]
EMS Lactobacillus_fermentum VelvetOptimizer 9
[nan, nan]
EMS Lactobacillus_fermentum VelvetOptimizer 10
[nan, nan]
EMS Lactobacillus_fermentum Unicycler 0
[0.0, 0.0]
EMS Lactobacillus_fermentum Unicycler 1
[nan, nan]
EMS Lactobacillus_fermentum Unicycler 2
[nan, nan]
EMS Lactobacillus_fermentum Un

EMS Escherichia_coli_plasmid MINIA 7
[nan, nan]
EMS Escherichia_coli_plasmid MINIA 8
[nan, nan]
EMS Escherichia_coli_plasmid MINIA 9
[nan, nan]
EMS Escherichia_coli_plasmid MINIA 10
[nan, nan]
EMS Escherichia_coli_plasmid metaSPAdes 0
[0.0, 0.0]
EMS Escherichia_coli_plasmid metaSPAdes 1
[1.0, 1.0]
EMS Escherichia_coli_plasmid metaSPAdes 2
[nan, nan]
EMS Escherichia_coli_plasmid metaSPAdes 3
[nan, nan]
EMS Escherichia_coli_plasmid metaSPAdes 4
[nan, nan]
EMS Escherichia_coli_plasmid metaSPAdes 5
[nan, nan]
EMS Escherichia_coli_plasmid metaSPAdes 6
[nan, nan]
EMS Escherichia_coli_plasmid metaSPAdes 7
[nan, nan]
EMS Escherichia_coli_plasmid metaSPAdes 8
[nan, nan]
EMS Escherichia_coli_plasmid metaSPAdes 9
[nan, nan]
EMS Escherichia_coli_plasmid metaSPAdes 10
[nan, nan]
EMS Escherichia_coli_plasmid MEGAHIT 0
[0.0, 0.0]
EMS Escherichia_coli_plasmid MEGAHIT 1
[0.0, 0.0]
EMS Escherichia_coli_plasmid MEGAHIT 2
[nan, nan]
EMS Escherichia_coli_plasmid MEGAHIT 3
[nan, nan]
EMS Escherichia_coli_pl

EMS Enterococcus_faecalis SPAdes 10
[nan, nan]
EMS Enterococcus_faecalis SKESA 0
[0.0, 0.0]
EMS Enterococcus_faecalis SKESA 1
[36.0, 36.0]
EMS Enterococcus_faecalis SKESA 2
[nan, nan]
EMS Enterococcus_faecalis SKESA 3
[nan, nan]
EMS Enterococcus_faecalis SKESA 4
[nan, nan]
EMS Enterococcus_faecalis SKESA 5
[nan, nan]
EMS Enterococcus_faecalis SKESA 6
[nan, nan]
EMS Enterococcus_faecalis SKESA 7
[nan, nan]
EMS Enterococcus_faecalis SKESA 8
[nan, nan]
EMS Enterococcus_faecalis SKESA 9
[nan, nan]
EMS Enterococcus_faecalis SKESA 10
[nan, nan]
EMS Enterococcus_faecalis MINIA 0
[0.0, 0.0]
EMS Enterococcus_faecalis MINIA 1
[nan, nan]
EMS Enterococcus_faecalis MINIA 2
[nan, nan]
EMS Enterococcus_faecalis MINIA 3
[nan, nan]
EMS Enterococcus_faecalis MINIA 4
[nan, nan]
EMS Enterococcus_faecalis MINIA 5
[nan, nan]
EMS Enterococcus_faecalis MINIA 6
[nan, nan]
EMS Enterococcus_faecalis MINIA 7
[nan, nan]
EMS Enterococcus_faecalis MINIA 8
[nan, nan]
EMS Enterococcus_faecalis MINIA 9
[nan, nan]
EMS E

,Reference,Assembler,Sample,Average,Upper bound,Lower bound,Lx
0,Staphylococcus_aureus_plasmid3,Unicycler,LNN,0.0,0.0,0.0,0
1,Staphylococcus_aureus_plasmid3,Unicycler,LNN,NaN,NaN,NaN,1
2,Staphylococcus_aureus_plasmid3,Unicycler,LNN,NaN,NaN,NaN,2
3,Staphylococcus_aureus_plasmid3,Unicycler,LNN,NaN,NaN,NaN,3
4,Staphylococcus_aureus_plasmid3,Unicycler,LNN,NaN,NaN,NaN,4
...,...,...,...,...,...,...,...
7519,Bacillus_subtilis,BCALM2,EMS,NaN,NaN,NaN,6
7520,Bacillus_subtilis,BCALM2,EMS,NaN,NaN,NaN,7
7521,Bacillus_subtilis,BCALM2,EMS,NaN,NaN,NaN,8
7522,Bacillus_subtilis,BCALM2,EMS,NaN,NaN,NaN,9


### Plot data

In [ ]:
for sample in sorted(plot_df['Sample'].unique(), key=lambda v: v.upper(), reverse=True):
    print('-' + sample)
    fig=make_subplots(rows=6, cols=2, column_titles=sorted(df['Reference'].unique(), key=lambda v: v.upper(), reverse=True),
                     shared_yaxes=True, shared_xaxes=True, x_title="Lx", y_title="Contigs")
    row=1
    col=1
    for reference in sorted(plot_df['Reference'].unique(), key=lambda v: v.upper(), reverse=True):
        print('--' + reference)
        i=0
        print(row, col)
        for assembler in sorted(plot_df['Assembler'].unique(), key=lambda v: v.upper(), reverse=True):
            print('---' + assembler)
            showlegend=True if col==1 and row==1 else False
            x = list(plot_df['Lx'][(plot_df['Assembler'] == assembler) & (plot_df['Reference'] == reference) & (plot_df['Sample'] == sample)])
            if x:
                y=list(plot_df['Average'][(plot_df['Assembler'] == assembler) & (plot_df['Reference'] == reference) & (plot_df['Sample'] == sample)])
                up_bound=list(plot_df['Upper bound'][(plot_df['Assembler'] == assembler) & (plot_df['Reference'] == reference) & (plot_df['Sample'] == sample)])
                low_bound=list(plot_df['Lower bound'][(plot_df['Assembler'] == assembler) & (plot_df['Reference'] == reference) & (plot_df['Sample'] == sample)])

                fig.add_trace(go.Scatter(name=assembler,
                                         x=x,
                                         y=y,
                                         mode='lines',
                                         line=dict(color=COLOURS[i]),
                                         showlegend=showlegend), col=col, row=row)
                fig.add_trace(go.Scatter(name='Upper Bound',
                                         x=x,
                                         y=up_bound,
                                         mode='lines',
                                         marker=dict(color=COLOURS[i]),line=dict(width=0),showlegend=False),col=col, row=row)
                fig.add_trace(go.Scatter(name='Lower Bound',
                                         x=x,
                                         y=low_bound,
                                         marker=dict(color=COLOURS[i]),
                                         line=dict(width=0),
                                         mode='lines',
                                         fillcolor=COLOURS[i],
                                         fill='tonexty',showlegend=False),col=col, row=row)
            i+=1
        if col == 2:
            col = 1
            row += 1
        else:
            col += 1
    fig.update_layout(plot_bgcolor='rgb(255,255,255)', title="{} Lx metric variation".format(sample))
    fig.update_xaxes(showline=True, linewidth=1, linecolor='#DCDCDC', gridcolor='#DCDCDC', rangemode='tozero')
    fig.update_yaxes(showline=True, linewidth=1, linecolor='#DCDCDC', gridcolor='#DCDCDC')
    for i in fig['layout']['annotations']:
        i['font']['size'] = 12
    fig.show()
    plot(fig, filename='Plots/Contiguity/Lx/{}.html'.format(sample), auto_open=False)

## NAx Metric

### Load data

In [ ]:
_files = glob.glob('../Results/*/results/*/stats/*nax.csv')

df_list = []
for f in _files:
    run = f.split('/')[-5]
    sample = f.split('/')[-3]
    assembler = f.split('/')[-1].split('_')[1]
    df = pd.read_csv(f)
    df['run'] = run
    df['Sample'] = sample
    df['Assembler'] = assembler
    df_list.append(df)

df = pd.concat(df_list)
df['distribution'] = np.where(df['Sample'].isin(log_distributed), 'Log', 'Even')
df

### Create Average, Upper bound and Lower bound variables for plot

In [ ]:
plot_df = pd.DataFrame(columns=['Reference', 'Assembler', 'Sample', "Average", "Upper bound", "Lower bound", 'NAx'])

for sample in sorted(df['Sample'].unique(), key=lambda v: v.upper(), reverse=True):
    for reference in sorted(df['Reference'].unique(), key=lambda v: v.upper(), reverse=True):
        for assembler in sorted(df['Assembler'].unique(), key=lambda v: v.upper(), reverse=True):
            for x in df['NAx'].unique():
                contigs = list(df['Basepairs'][(df['Assembler'] == assembler) & (df['Reference'] == reference) & (df['Sample'] == sample) & (df['NAx'] == x)])
                if contigs:
                    avg = mean(contigs) if len(set(contigs)) > 1 else contigs[0]
                    upper_bound = max(contigs)
                    lower_bound = min(contigs)
                    plot_df = plot_df.append({'Reference':reference, 'Assembler': assembler, 'Sample': sample, "Average": avg, "Upper bound": upper_bound, "Lower bound": lower_bound, 'NAx': x}, ignore_index=True)

plot_df

### Plot data

In [ ]:
for sample in sorted(plot_df['Sample'].unique(), key=lambda v: v.upper(), reverse=True):
    print('-' + sample)
    fig=make_subplots(rows=6, cols=2, column_titles=sorted(df['Reference'].unique(), key=lambda v: v.upper(), reverse=True),
                     shared_yaxes=True, shared_xaxes=True, x_title="NAx", y_title="Basepairs")
    row=1
    col=1
    for reference in sorted(plot_df['Reference'].unique(), key=lambda v: v.upper(), reverse=True):
        print('--' + reference)
        i=0
        print(row, col)
        for assembler in sorted(plot_df['Assembler'].unique(), key=lambda v: v.upper(), reverse=True):
            print('---' + assembler)
            showlegend=True if col==1 and row==1 else False
            x = list(plot_df['NAx'][(plot_df['Assembler'] == assembler) & (plot_df['Reference'] == reference) & (plot_df['Sample'] == sample)])
            if x:
                y=list(plot_df['Average'][(plot_df['Assembler'] == assembler) & (plot_df['Reference'] == reference) & (plot_df['Sample'] == sample)])
                up_bound=list(plot_df['Upper bound'][(plot_df['Assembler'] == assembler) & (plot_df['Reference'] == reference) & (plot_df['Sample'] == sample)])
                low_bound=list(plot_df['Lower bound'][(plot_df['Assembler'] == assembler) & (plot_df['Reference'] == reference) & (plot_df['Sample'] == sample)])
                print(x)
                print(y)
                print(up_bound)
                print(low_bound)

                fig.add_trace(go.Scatter(name=assembler,
                                         x=x,
                                         y=y,
                                         mode='lines',
                                         line=dict(color=COLOURS[i]),
                                         showlegend=showlegend), col=col, row=row)
                fig.add_trace(go.Scatter(name='Upper Bound',
                                         x=x,
                                         y=up_bound,
                                         mode='lines',
                                         marker=dict(color=COLOURS[i]),line=dict(width=0),showlegend=False),col=col, row=row)
                fig.add_trace(go.Scatter(name='Lower Bound',
                                         x=x,
                                         y=low_bound,
                                         marker=dict(color=COLOURS[i]),
                                         line=dict(width=0),
                                         mode='lines',
                                         fillcolor=COLOURS[i],
                                         fill='tonexty',showlegend=False),col=col, row=row)
            i+=1
        if col == 2:
            col = 1
            row += 1
        else:
            col += 1
    fig.update_layout(plot_bgcolor='rgb(255,255,255)', title="{} NAx metric variation".format(sample))
    fig.update_xaxes(showline=True, linewidth=1, linecolor='#DCDCDC', gridcolor='#DCDCDC', rangemode='tozero')
    fig.update_yaxes(showline=True, linewidth=1, linecolor='#DCDCDC', gridcolor='#DCDCDC')
    for i in fig['layout']['annotations']:
        i['font']['size'] = 12
    fig.show()
    plot(fig, filename='Plots/Contiguity/NAx/{}.html'.format(sample), auto_open=False)

## NGx Metric

### Load data

In [ ]:
_files = glob.glob('../Results/*/results/*/stats/*ngx.csv')

df_list = []
for f in _files:
    run = f.split('/')[-5]
    sample = f.split('/')[-3]
    assembler = f.split('/')[-1].split('_')[1]
    df = pd.read_csv(f)
    df['run'] = run
    df['Sample'] = sample
    df['Assembler'] = assembler
    df_list.append(df)

df = pd.concat(df_list)
df['distribution'] = np.where(df['Sample'].isin(log_distributed), 'Log', 'Even')
df

### Create Average, Upper bound and Lower bound variables for plot

In [ ]:
plot_df = pd.DataFrame(columns=['Reference', 'Assembler', 'Sample', "Average", "Upper bound", "Lower bound", 'NGx'])

for sample in sorted(df['Sample'].unique(), key=lambda v: v.upper(), reverse=True):
    for reference in sorted(df['Reference'].unique(), key=lambda v: v.upper(), reverse=True):
        for assembler in sorted(df['Assembler'].unique(), key=lambda v: v.upper(), reverse=True):
            for x in df['NGx'].unique():
                contigs = list(df['Basepairs'][(df['Assembler'] == assembler) & (df['Reference'] == reference) & (df['Sample'] == sample) & (df['NGx'] == x)])
                if contigs:
                    avg = mean(contigs) if len(set(contigs)) > 1 else contigs[0]
                    upper_bound = max(contigs)
                    lower_bound = min(contigs)
                    plot_df = plot_df.append({'Reference':reference, 'Assembler': assembler, 'Sample': sample, "Average": avg, "Upper bound": upper_bound, "Lower bound": lower_bound, 'NGx': x}, ignore_index=True)

plot_df

### Plot data

In [ ]:
for sample in sorted(plot_df['Sample'].unique(), key=lambda v: v.upper(), reverse=True):
    print('-' + sample)
    fig=make_subplots(rows=6, cols=2, column_titles=sorted(df['Reference'].unique(), key=lambda v: v.upper(), reverse=True),
                     shared_yaxes=True, shared_xaxes=True, x_title="NGx", y_title="Basepairs")
    row=1
    col=1
    for reference in sorted(plot_df['Reference'].unique(), key=lambda v: v.upper(), reverse=True):
        print('--' + reference)
        i=0
        print(row, col)
        for assembler in sorted(plot_df['Assembler'].unique(), key=lambda v: v.upper(), reverse=True):
            print('---' + assembler)
            showlegend=True if col==1 and row==1 else False
            x = list(plot_df['NGx'][(plot_df['Assembler'] == assembler) & (plot_df['Reference'] == reference) & (plot_df['Sample'] == sample)])
            if x:
                y=list(plot_df['Average'][(plot_df['Assembler'] == assembler) & (plot_df['Reference'] == reference) & (plot_df['Sample'] == sample)])
                up_bound=list(plot_df['Upper bound'][(plot_df['Assembler'] == assembler) & (plot_df['Reference'] == reference) & (plot_df['Sample'] == sample)])
                low_bound=list(plot_df['Lower bound'][(plot_df['Assembler'] == assembler) & (plot_df['Reference'] == reference) & (plot_df['Sample'] == sample)])

                fig.add_trace(go.Scatter(name=assembler,
                                         x=x,
                                         y=y,
                                         mode='lines',
                                         line=dict(color=COLOURS[i]),
                                         showlegend=showlegend), col=col, row=row)
                fig.add_trace(go.Scatter(name='Upper Bound',
                                         x=x,
                                         y=up_bound,
                                         mode='lines',
                                         marker=dict(color=COLOURS[i]),line=dict(width=0),showlegend=False),col=col, row=row)
                fig.add_trace(go.Scatter(name='Lower Bound',
                                         x=x,
                                         y=low_bound,
                                         marker=dict(color=COLOURS[i]),
                                         line=dict(width=0),
                                         mode='lines',
                                         fillcolor=COLOURS[i],
                                         fill='tonexty',showlegend=False),col=col, row=row)
            i+=1
        if col == 2:
            col = 1
            row += 1
        else:
            col += 1
    fig.update_layout(plot_bgcolor='rgb(255,255,255)', title="{} NGx metric variation".format(sample))
    fig.update_xaxes(showline=True, linewidth=1, linecolor='#DCDCDC', gridcolor='#DCDCDC', rangemode='tozero')
    fig.update_yaxes(showline=True, linewidth=1, linecolor='#DCDCDC', gridcolor='#DCDCDC')
    for i in fig['layout']['annotations']:
        i['font']['size'] = 12
    fig.show()
    plot(fig, filename='Plots/Contiguity/NGx/{}.html'.format(sample), auto_open=False)